In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from financerag.tasks import FinDER
from torch.utils.data import DataLoader


finder_task = FinDER()

df = pd.read_csv('./tsv/FinDER_qrels.tsv', sep='\t')

queries_map = finder_task.queries
corpus_map = finder_task.corpus
queries = df['query_id'].apply(lambda x: queries_map.get(x, None))
corpus = df['corpus_id'].apply(lambda x: corpus_map.get(x, None))
train_samples = [
    InputExample(texts=[query, doc], label=1)
    for query, doc in zip(queries, corpus)
]
model = SentenceTransformer('all-MiniLM-L6-v2')
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=4,  
    warmup_steps=100,
    weight_decay=0.01,
)
model.save("fine_tuned_model")
df.to_pickle('dataframe.pkl')





In [7]:
[
    [query, doc]
    for query, doc in zip(queries, corpus)
]